### Image to image

In [1]:
import os
import torch
import pandas as pd
from PIL import Image

from engine_img2img import (get_model_info, 
                    ImgToImg, 
                    get_single_text_embedding, 
                    get_single_image_embedding, 
                    get_top_N_images, 
                    plot_images_by_side, 
                    get_total_embds,
                    getData)

c:\Users\alex\Desktop\service\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
EKB_places = pd.read_csv("data/EKB_places_pure.csv")
NN_places = pd.read_csv("data/NN_places_pure.csv")
VLD_places = pd.read_csv("data/VLAD_places_pure.csv")
YAR_places = pd.read_csv("data/YAROSLAVL_places_pure.csv")

data_places = [EKB_places, NN_places, VLD_places, YAR_places]

In [3]:
model_ID = "openai/clip-vit-base-patch32"
model, processor, tokenizer = get_model_info(model_ID)

model = ImgToImg()

if not os.path.exists("data/EKB_images_emb.csv"):
    print ('Create')
    EKB_img = pd.read_csv("data/EKB_images.csv")
    # NN_img = pd.read_csv("data/NN_images.csv")
    # VLD_img = pd.read_csv("data/Vladimir_images.csv")
    # YAR_img = pd.read_csv("data/Yaroslavl_images.csv")

    # data_img = [EKB_img, NN_img, VLD_img, YAR_img]
    data_img = [EKB_img]
    model.dataToJpg_(data_img, "image")

    model.get_total_embds_(EKB_img, EKB_places, "pil", "Kind")
    # model.get_total_embds_(NN_img, NN_places, "pil", "Kind")
    # model.get_total_embds_(VLD_img, VLD_places, "pil", "Kind")
    # model.get_total_embds_(YAR_img, YAR_places, "pil", "Kind")

    EKB_img.to_csv("data/EKB_images_emb.csv")
    # NN_img.to_csv("data/NN_images_emb.csv")
    # VLD_img.to_csv("data/Vladimir_images_emb.csv")
    # YAR_img.to_csv("data/Yaroslavl_images_emb.csv")

else:
    print ('Load')
    EKB_img = pd.read_csv("data/EKB_images_emb.csv")
    # NN_img = pd.read_csv("data/NN_images_emb.csv")
    # VLD_img = pd.read_csv("data/Vladimir_images_emb.csv")

    # YAR_img = pd.read_csv("data/Yaroslavl_images_emb.csv")

Load


In [4]:
EKB_img.head()

,Unnamed: 0,name,image,pil,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,...,embedding_502,embedding_503,embedding_504,embedding_505,embedding_506,embedding_507,embedding_508,embedding_509,embedding_510,embedding_511
0,0,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,0.288855,0.264261,-0.035371,-0.003198,-0.023572,-0.389865,...,-0.220806,0.005686,-0.451456,0.028694,0.320810,0.231149,-0.505051,-0.148276,0.200847,-0.103384
1,1,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,0.351544,0.494194,-0.071660,0.171966,0.044355,-0.207105,...,-0.403729,0.024200,0.353770,0.524241,-0.023235,0.390095,0.070125,-0.169580,-0.325563,-0.388260
2,2,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,0.133327,0.158181,-0.539799,-0.124352,0.068363,-0.250706,...,0.383440,0.308557,0.030198,0.458824,0.115400,0.252704,0.140298,-0.143117,-0.014858,-0.095148
3,3,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,0.187970,0.181815,-0.274064,-0.001178,0.098211,-0.291843,...,0.279845,0.422315,0.140452,0.600820,0.212584,0.456396,-0.000368,-0.093129,-0.169146,-0.013710
4,4,Динамо,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,0.058530,0.173466,-0.534769,0.175547,0.560623,-0.324205,...,-0.063911,0.292164,0.531405,0.318937,0.038730,0.303230,0.142911,0.266096,-0.164265,-0.175399


In [5]:
query_image = os.listdir('example')[0]
query_image = Image.open(os.path.join('example', query_image))

In [12]:
top_images = model.get_top_N_images_(query_image, EKB_img, EKB_places, search_criterion="image" )
top_images

data  = getData(top_images,EKB_places)
data.head()

c:\Users\alex\Desktop\service\engine_img2img.py:364: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, entries])


,WikiData,Name,Kind,City,Rate,Lon,Lat,russian_captions
26,Q28381390,Дом Гайдара,"historic_architecture,architecture,interesting...",Екатеринбург,3h,60.591591,56.845341,историческая архитектура архитектура интересны...
49,Q4306077,Литературная жизнь Урала XX века,"cultural,museums,interesting_places,other_museums",Екатеринбург,3h,60.606716,56.842705,культурные музеи интересные места другие музеи
48,Q4306075,Музей «Литературная жизнь Урала XIX века»,"museums,cultural,children_museums,interesting_...",Екатеринбург,3h,60.607609,56.843414,музеи культурные детские музеи интересные места
21,Q27888113,Дом Э. Ф. Филитц,"historic_architecture,architecture,interesting...",Екатеринбург,3h,60.620441,56.833843,историческая архитектура архитектура интересны...
61,Q4474328,улица Февральской Революции,"cultural,urban_environment,interesting_places,...",Екатеринбург,3h,60.591040,56.842146,культурные городская среда интересные места пл...


### Text to image

In [1]:
import io
import base64
import pandas as pd
from PIL import Image 

from engine_text2img import RuClipTopK, getData

c:\Users\alex\Desktop\service\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
EKB_img = pd.read_csv("data/EKB_images.csv")
EKB_places = pd.read_csv("data/EKB_places_pure.csv")

In [3]:
templates = ['{}', 'это {}', 'на картинке {}']
model = RuClipTopK("cuda", 10, templates)
images = [Image.open(io.BytesIO(base64.b64decode(i))) for i in EKB_img["image"]]
EKB_img["image"] = images
EKB_img = model.get_total_embds_(EKB_img, EKB_places, "image", "russian_captions")

0it [00:00, ?it/s]c:\Users\alex\Desktop\service\venv\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
1it [00:00,  1.33it/s]
1it [00:00, 62.33it/s]
1it [00:00, 62.16it/s]
1it [00:00, 65.57it/s]
1it [00:00, 65.55it/s]
1it [00:00, 70.18it/s]
1it [00:00, 72.52it/s]
1it [00:00, 67.91it/s]
1it [00:00, 62.34it/s]
1it [00:00, 66.49it/s]
1it [00:00, 62.34it/s]
1it [00:00, 60.91it/s]
1it [00:00, 55.71it/s]
1it [00:00, 54.83it/s]
1it [00:00, 62.95it/s]
1it [00:00, 62.34it/s]
1it [00:00, 55.14it/s]
1it [00:00, 58.67it/s]
1it [00:00, 55.75it/s]
1it [00:00, 55.41it/s]
1it [00:00, 59.07it/s]
1it [00:00, 62.34it/s]
1it [00:00, 59.13it/s]
1it [00:00, 51.19it/s]
1it [00:00, 58.67it/s]
1it [00:00, 58.67it/s]
1it [00:00, 66.49it/s]
1it [00:00, 52.49it/s]
1it [00:00, 61.77it/s]


In [4]:
top_images = model.get_top_N_images_("церкви", EKB_img)
data = getData(top_images, EKB_places)
data.head()

c:\Users\alex\Desktop\service\engine_text2img.py:239: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, entries])


,WikiData,Name,Kind,City,Rate,Lon,Lat,russian_captions,captions_embds
86,Q631979,Храм на Крови,"religion,churches,interesting_places,eastern_o...",Екатеринбург,3h,60.608309,56.844339,религиозное церкви интересные места восточные ...,"[[0.27842385, 0.40294808, -0.6540004, -0.04114..."
41,Q4202547,Иоанно-Предтеченский Кафедральный собор,"religion,cathedrals,churches,interesting_place...",Екатеринбург,3h,60.574745,56.829426,религиозное соборы церкви интересные места вос...,"[[0.32465544, 0.47960305, -0.75125414, -0.0089..."
72,Q4500740,Храм в честь Вознесения Господня,"religion,churches,interesting_places,eastern_o...",Екатеринбург,3h,60.612843,56.844761,религиозное церкви интересные места восточные ...,"[[0.27842385, 0.40294808, -0.6540004, -0.04114..."
44,Q4240385,Крестовоздвиженская церковь (Екатеринбург),"religion,churches,interesting_places,eastern_o...",Екатеринбург,3h,60.622898,56.831982,религиозное церкви интересные места восточные ...,"[[0.27842385, 0.40294808, -0.6540004, -0.04114..."
28,Q3935233,Преображенская церковь на Уктусе,"religion,churches,interesting_places,eastern_o...",Екатеринбург,3h,60.649147,56.780872,религиозное церкви интересные места восточные ...,"[[0.27842385, 0.40294808, -0.6540004, -0.04114..."
